<a href="https://colab.research.google.com/github/MalihaUCF/Machine-Learning-Course-Assignments-Spring-2019/blob/master/Assignment%202/Final_k_fold_Model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Loading CIFAR-10 dataset

from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
import numpy as np
import keras


#creating train,validate and test sets
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images1, val_images1, train_labels1, val_labels1 = train_test_split(train_images,train_labels,test_size=0.0002,stratify=train_labels)

print(train_images.shape)
print(train_images1.shape)
print(val_images1.shape)
print(test_images.shape)
print(train_labels1.shape)


for i in range(0,50):
  print(train_labels[i])

def k_fold_dataset(i,train_images1,train_labels1):
  val_index_a=i*10000
  val_index_b=val_index_a+10000
  
  #splitting train images into train and val set
  #val set
  X_val=train_images1[val_index_a:val_index_b]
  y_val=train_labels1[val_index_a:val_index_b]
  
  #train set
  X_train=np.append(train_images1[0:val_index_a],train_images1[val_index_b:50000],axis=0)
  y_train=np.append(train_labels1[0:val_index_a],train_labels1[val_index_b:50000],axis=0)
  
  
  return X_train,y_train,X_val,y_val
  


In [0]:
batch_size = 32
epochs = 30
data_augmentation = False
num_predictions = 20

from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D


#using our best model with k-fold
def train_model1(i,X_train,y_train,X_val,y_val,test_images,test_labels):
  

  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',
                   input_shape=X_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))
  
 

  #using ADAM

  Adam=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)


  # Let's train our best model using Adam
  model.compile(loss='categorical_crossentropy',
                optimizer=Adam,
                metrics=['accuracy'])



  if not data_augmentation:
      print('Not using data augmentation.')
      history=model.fit(X_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(X_val, y_val),
                shuffle=True)
  else:
      print('Using real-time data augmentation.')
      # This will do preprocessing and realtime data augmentation:
       datagen = ImageDataGenerator(
      rotation_range=15,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True)
    
      datagen.fit(X_train)

      
      # Fit the model on the batches generated by datagen.flow().
      model.fit_generator(datagen.flow(X_train, y_train,
                                       batch_size=batch_size),
                          epochs=epochs,steps_per_epoch=100,validation_steps=50,
                          validation_data=(X_val, y_val))

  
  
  from sklearn.metrics import classification_report, confusion_matrix
  import os
  from keras.models import load_model


  y_pred=model.predict(test_images) 
  y_pred = [np.argmax(entry) for entry in y_pred]
  y_true = [np.argmax(entry) for entry in test_labels]

  print('Confusion Matrix\n\n')
  print(confusion_matrix(y_true, y_pred))  

  print(classification_report(y_pred,y_true))

In [3]:
from random import shuffle

num_classes=10
# Convert class vectors to binary class matrices.
test_labels = keras.utils.to_categorical(test_labels, num_classes)


for a in range(0,5):
    X_train,y_train,X_val,y_val=k_fold_dataset(a,train_images1,train_labels1)
    
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    X_train /= 255
    X_val /= 255
    
    
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_val=keras.utils.to_categorical(y_val, num_classes)
    
    print(train_images1.shape)
    print(X_train.shape)
    print(X_val.shape)
    print(test_images.shape)  
    print(y_val.shape)
    
    train_model1(a,X_train,y_train,X_val,y_val,test_images, test_labels)


(49990, 32, 32, 3)
(39990, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 10)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Not using data augmentation.
Instructions for updating:
Use tf.cast instead.
Train on 39990 samples, validate on 10000 samples
Epoch 1/30
39990/39990 [==============================] - 35s 865us/step - loss: 1.8311 - acc: 0.3266 - val_loss: 1.5295 - val_acc: 0.4526
Epoch 2/30
39990/39990 [==============================] - 31s 770us/step - loss: 1.4999 - acc: 0.4556 - val_loss: 1.3944 - val_acc: 0.5027
Epoch 3/30
39990/39990 [==============================] - 31s 775us/step - loss: 1.3786 - acc: 0.5043 - val_loss: 1.2788 - val_acc: 0.5439
Epoch 4/30
39990/39990 [==============================] - 31s 778us/step - loss: 1.2862 - acc: 0.5389 - val_loss: 1.1894 - val_acc: 0.5797
Epoch 5/30
39990/39990 [==================

In [0]:
# average of k-fold accuracy as seen above in the classification reports

k1=65
k2=65
k3=60
k4=65
k5=63


#Saving the model as an h5 file

good_model='adam_dropout_k-fold_out.h5'
model.save(good_model)

from google.colab import files

with open(good_model, 'r') as f:
  files.download(good_model)

In [11]:
#printing the f1, microavg scores per fold

print('k1 Test accuracy : ', k1,'%')
print('k2 Test accuracy : ', k2,'%')
print('k3 Test accuracy : ', k3,'%')
print('k4 Test accuracy : ', k4,'%')
print('k5 Test accuracy : ', k5,'%')

avg=(k1+k2+k3+k4+k5)/5
print('\n')
print('Avg score using k fold validation is ' ,avg,'% after 30 epochs for each training and val set')

k1 Test accuracy :  65 %
k2 Test accuracy :  65 %
k3 Test accuracy :  60 %
k4 Test accuracy :  65 %
k5 Test accuracy :  63 %


Avg score using k fold validation is  63.6 % after 30 epochs for each training and val set


In [0]:
!ls

adam_dropout_hold_out.h5  sample_data
